In [1]:
import numpy as np
import pandas as pd
import json
import itertools
from ast import literal_eval

<h2>Tuning the # of candidate items<h2>

In [22]:
# filename = f"/home/diego/chat-reranking/experiments/anime/itemid_to_name.pkl"
# with open(f"{filename}", 'rb') as fp:
#     itemid_to_name = pickle.load(fp)
# filename = f"/home/diego/chat-reranking/experiments/anime/itemname_to_id.pkl"
# with open(f"{filename}", 'rb') as fp:
#     itemname_to_id = pickle.load(fp)

In [23]:
# for i in [16498,31964,30276,20583,32182,33352,24833,34599,13601]:
#     print(f"{i}: {itemid_to_name[i]}")
# itemname_to_id["Assassination Classroom Second Season"]

In [11]:
recs_folder = "/home/diego/chat-reranking/experiments/anime/recs/reranked"
recs_name = "chatgpt0613-div-p1-pzt-fold_0_50_20"
top_n = 10

# read JSON file
f = open(f"{recs_folder}/{recs_name}.json")
data = json.load(f)
f.close()

In [12]:
print("Tune the length of the candidate set: chatgpt")
max_pos = []
for entry in data:
    base = entry['recs']
    re_ranked = entry['reranked_recs']
    if len(re_ranked) == 0:
        continue

    def return_pos(j):
        if j in base:
            return base.index(j)
        else:
            return 0

    pos = [return_pos(r) for r in re_ranked]
    max_pos.append(max(pos))

print(f"Average max reranking pos: {np.mean(max_pos)}") 
print(f"Average std reranking pos: {np.std(max_pos)}")   

Tune the length of the candidate set: chatgpt
Average max reranking pos: 34.06666666666667
Average std reranking pos: 15.167802155296668


In [10]:
rerankers = ["MMR-pzt-fold_0_50_20", "RxQuAD-pzt-fold_0_50_20", "xQuAD-pzt-fold_0_50_20", "Random-pzt-fold_0_50_20"]

print(f"Tune the length of the candidate set: baseline reranker {rerankers[3]}")
base_recs = pd.read_csv(f"/home/diego/chat-reranking/experiments/anime/recs/baselines/pzt-fold_0_50_20", 
                            names=["userid", "itemid", "rating"], sep="\t")
re_ranked = pd.read_csv(f"/home/diego/chat-reranking/experiments/anime/recs/reranked/{rerankers[3]}", 
                            names=["userid", "itemid", "rating"], sep="\t")

max_pos = []
for userid in re_ranked["userid"].unique()[:300]:
    base = list(base_recs[base_recs["userid"] == userid]["itemid"].values[:50])
    recs = re_ranked[re_ranked["userid"] == userid]["itemid"].values.tolist()
        
    pos = [base.index(r) for r in recs]
    max_pos.append(max(pos))
print(f"Average max reranking pos: {np.mean(max_pos)}") 
print(f"Average std reranking pos: {np.std(max_pos)}")  

Tune the length of the candidate set: baseline reranker Random-pzt-fold_0_50_20
Average max reranking pos: 44.95666666666666
Average std reranking pos: 3.8385920451239524


<h2>Convert rec files for Ranksys</h2>

In [2]:
recs_folder = "/home/diego/chat-reranking/experiments/anime/recs/reranked_final/"
recs_name = "Llama-3-8b-instruct-div-p47-pzt-fold_0"
top_n = 10

# read JSON file
f = open(f"{recs_folder}/{recs_name}.json")
data = json.load(f)
f.close()

In [3]:
train_folder = f"/home/diego/chat-reranking/experiments/anime/fold_0/train_data.csv"
training_data = pd.read_csv(f'{train_folder}', names=["userid", "itemid", "rating"], sep="\t")

Some recommendations might contain 
- less than 10 items
- items that are from the training

We need to remove these items from the recommenations (and count them)

In [4]:
# check whether some recommendations don't intersect with the candidate set
recs = {}
candidate_set = {}
tot_rec_before_pruning = 0
tot_rec_after_pruning = 0
for entry in data:
    userid = entry['userid']
    recs[userid] = entry['reranked_recs']
    candidate_set[userid] = entry['recs']
    tot_rec_before_pruning += len(recs[userid])
    user_train = training_data[training_data["userid"] == entry['userid']]["itemid"].values.tolist()
    to_prune = recs[userid]
    for i, item in enumerate(recs[userid]):
        if item not in candidate_set[userid]:
            del to_prune[i]
            # print(f"user {entry['userid']}: {item}")
            # print(item)
            # print(entry['recs'])
            # print("######")
    recs[userid] = to_prune
    tot_rec_after_pruning += len(to_prune)
print(f"# recs before pruning: {tot_rec_before_pruning}")
print(f"# recs after pruning: {tot_rec_after_pruning}")

# recs before pruning: 5000
# recs after pruning: 5000


In [5]:
# count users with less than 10 recommendations
count = 0
count_none = 0
avg_len = []
for userid in recs:
    avg_len.append(len(recs[userid]))
    if len(recs[userid]) < 10:
        count += 1
    if len(recs[userid]) == 0:
        count_none += 1
print(f"# of users with less than 10 recommendations: {count}")
print(f"# of users with no recommendations: {count_none}")
print(f"avg number of recommendations per user: {np.mean(avg_len)}")

# of users with less than 10 recommendations: 0
# of users with no recommendations: 0
avg number of recommendations per user: 10.0


In [6]:
# print final recommendations
exclude_violating_recs = True
excluded = 0
with open(f"{recs_folder}/{recs_name}", "w") as f:
    for userid in recs:
        if len(recs[userid]) < 10:
            excluded += 1
            continue
        score = float(top_n)
        usr_str = ""
        for r in recs[userid]:
            usr_str += f"{userid}\t{r}\t{score}\n"
            score -= 1.0
        f.write(usr_str)
print("Printed!")
print(f"# of excluded users: {excluded}")

Printed!
# of excluded users: 0


<h2>Output tokens<h2>

In [14]:
# load raw_file
prompt_id = 5
input_file = f"/home/diego/chat-reranking/experiments/goodreads/recs/reranked_final/chatgpt0613-div-p{prompt_id}-pzt-fold_0.json"
f = open(input_file)
data = json.load(f)

tot_len = 0
for entry in data:
    tot_len += len(entry['raw_gpt_outputs'])
    # entry['raw_gpt_output'] = recs[entry['userid']]

print(tot_len)

277163


In [11]:
# load raw_file
prompt_id = 1
input_file = f"/home/diego/chat-reranking/experiments/goodreads/recs/reranked_final/instructgpt-div-p{prompt_id}-pzt-fold_0.json"
f = open(input_file)
data = json.load(f)

tot_len = 0
for entry in data:
    tot_len += len(entry['raw_gpt_outputs'])
    # entry['raw_gpt_output'] = recs[entry['userid']]

print(tot_len)

220780


<h2>Results anime (dataset features)</h2>

In [123]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
baseline_recs = pd.read_csv(f"{results_path}/pzt-fold_0-@10.csv")
random_recs = pd.read_csv(f"{results_path}/Random-pzt-fold_0-@10.csv")
# gpt_recs = pd.read_csv(f"{results_path}/chatgpt0613-div-p1-pzt-fold_0-@10.csv")
mmr_recs = pd.read_csv(f"{results_path}/MMR-pzt-fold_0-@10.csv")
xquad_recs = pd.read_csv(f"{results_path}/xQuAD-pzt-fold_0-@10.csv")
rxquad_recs = pd.read_csv(f"{results_path}/RxQuAD-pzt-fold_0-@10.csv")

In [126]:
d = {"reranker": ["baseline (MF)", "Random", "MMR", "xQuAD", "RxQuAD"]}
# print("ChatGPT better than baseline in the following metrics:")
for col in baseline_recs.columns:
    d[col] = [baseline_recs[col][0], random_recs[col][0], mmr_recs[col][0], xquad_recs[col][0], rxquad_recs[col][0]]
    # if gpt_recs[col][0] > baseline_recs[col][0]:
    #     print(col)
results = pd.DataFrame(data=d)

In [127]:
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["baseline (MF)", "Random", "MMR", "xQuAD", "RxQuAD"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2914,0.316022,0.324519,0.221073,0.775894,0.23365,0.47285,baseline (MF)
1,0.1468,0.130343,0.153897,0.094422,0.810569,0.14475,0.49925,Random
2,0.2720,0.298836,0.313375,0.221876,0.874003,0.23715,0.54365,MMR
3,0.2830,0.301404,0.350005,0.211741,0.779803,0.25115,0.53890,xQuAD
4,0.2796,0.295440,0.337932,0.206974,0.769586,0.24370,0.50615,RxQuAD


In [133]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
chat_p1 = pd.read_csv(f"{results_path}/chatgpt0613-div-p1-pzt-fold_0-@10.csv")
chat_p2 = pd.read_csv(f"{results_path}/chatgpt0613-div-p2-pzt-fold_0-@10.csv")
chat_p3 = pd.read_csv(f"{results_path}/chatgpt0613-div-p3-pzt-fold_0-@10.csv")
chat_p4 = pd.read_csv(f"{results_path}/chatgpt0613-div-p4-pzt-fold_0-@10.csv")
chat_p5 = pd.read_csv(f"{results_path}/chatgpt0613-div-p5-pzt-fold_0-@10.csv")
chat_p6 = pd.read_csv(f"{results_path}/chatgpt0613-div-p6-pzt-fold_0-@10.csv")

In [134]:
d = {}
for col in chat_p1.columns:
    d[col] = [chat_p1[col][0], 
              chat_p2[col][0], 
              chat_p3[col][0], 
              chat_p4[col][0],
              chat_p5[col][0],
              chat_p6[col][0],
              ]
results = pd.DataFrame(data=d)
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["chat-gpt-p1", "chat-gpt-p2", "chat-gpt-p3", "chat-gpt-p4", 
                       "chat-gpt-p5", "chat-gpt-p6"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2452,0.270062,0.286624,0.185838,0.812312,0.21485,0.50335,chat-gpt-p1
1,0.2710,0.295994,0.312172,0.211770,0.812039,0.23160,0.50230,chat-gpt-p2
2,0.2540,0.276861,0.299468,0.195332,0.815794,0.22390,0.51065,chat-gpt-p3
3,0.2302,0.251550,0.275403,0.173522,0.817197,0.20865,0.51405,chat-gpt-p4
4,0.2540,0.275740,0.296433,0.193743,0.813080,0.22125,0.50740,chat-gpt-p5
5,0.2506,0.272173,0.296205,0.193616,0.817579,0.22310,0.51355,chat-gpt-p6


In [135]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
instruct_p1 = pd.read_csv(f"{results_path}/instructgpt-div-p1-pzt-fold_0-@10.csv")
instruct_p2 = pd.read_csv(f"{results_path}/instructgpt-div-p2-pzt-fold_0-@10.csv")
instruct_p3 = pd.read_csv(f"{results_path}/instructgpt-div-p3-pzt-fold_0-@10.csv")
instruct_p4 = pd.read_csv(f"{results_path}/instructgpt-div-p4-pzt-fold_0-@10.csv")
instruct_p5 = pd.read_csv(f"{results_path}/instructgpt-div-p5-pzt-fold_0-@10.csv")
instruct_p6 = pd.read_csv(f"{results_path}/instructgpt-div-p6-pzt-fold_0-@10.csv")

d = {}
for col in chat_p1.columns:
    d[col] = [instruct_p1[col][0], 
              instruct_p2[col][0], 
              instruct_p3[col][0], 
              instruct_p4[col][0],
              instruct_p5[col][0],
              instruct_p6[col][0],
              ]
results = pd.DataFrame(data=d)
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["instruct-gpt-p1", "instruct-gpt-p2", "instruct-gpt-p3", "instruct-gpt-p4", 
                       "instruct-gpt-p5", "instruct-gpt-p6"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2430,0.262564,0.286618,0.185676,0.812275,0.21915,0.51010,instruct-gpt-p1
1,0.2454,0.272404,0.291284,0.186974,0.813067,0.21710,0.50565,instruct-gpt-p2
2,0.2372,0.260093,0.281940,0.181968,0.818466,0.21340,0.51370,instruct-gpt-p3
3,0.2394,0.257365,0.283150,0.181650,0.813911,0.21585,0.51060,instruct-gpt-p4
4,0.2466,0.270360,0.292655,0.188584,0.815421,0.21930,0.50480,instruct-gpt-p5
5,0.2462,0.274018,0.294508,0.188309,0.813147,0.21825,0.50360,instruct-gpt-p6


In [137]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
llama_p7 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p7-pzt-fold_0-@10.csv")
llama_p8 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p8-pzt-fold_0-@10.csv")
llama_p9 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p9-pzt-fold_0-@10.csv")
llama_p10 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p10-pzt-fold_0-@10.csv")
llama_p11 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p11-pzt-fold_0-@10.csv")
llama_p12 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p12-pzt-fold_0-@10.csv")

d = {}
for col in chat_p1.columns:
    d[col] = [llama_p7[col][0], 
              llama_p8[col][0], 
              llama_p9[col][0], 
              llama_p10[col][0],
              llama_p11[col][0],
              llama_p12[col][0],
              ]
results = pd.DataFrame(data=d)
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["llama-p7", "llama-p8", "llama-p9", "llama-p10", 
                       "llama-p11", "llama-p12"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2296,0.250901,0.277415,0.169752,0.812883,0.21020,0.51205,llama-p7
1,0.2106,0.233646,0.255733,0.156441,0.818012,0.19515,0.50245,llama-p8
2,0.2126,0.234732,0.259793,0.155878,0.818021,0.19620,0.51005,llama-p9
3,0.2318,0.253362,0.281120,0.173538,0.810288,0.21395,0.51190,llama-p10
4,0.2210,0.235086,0.269004,0.164583,0.806263,0.20665,0.51185,llama-p11
5,0.2232,0.241760,0.271037,0.166399,0.805794,0.20610,0.51135,llama-p12


<h2>Results anime (ChatGPT features)</h2>

In [139]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval_gpt_genres"
baseline_recs = pd.read_csv(f"{results_path}/pzt-fold_0-@10.csv")
random_recs = pd.read_csv(f"{results_path}/Random-pzt-fold_0-@10.csv")
mmr_recs = pd.read_csv(f"{results_path}/MMR-pzt-fold_0-@10.csv")
xquad_recs = pd.read_csv(f"{results_path}/xQuAD-pzt-fold_0-@10.csv")
rxquad_recs = pd.read_csv(f"{results_path}/RxQuAD-pzt-fold_0-@10.csv")

In [140]:
d = {"reranker": ["baseline (MF)", "Random", "MMR", "xQuAD", "RxQuAD"]}
# print("ChatGPT better than baseline in the following metrics:")
for col in baseline_recs.columns:
    d[col] = [baseline_recs[col][0], random_recs[col][0], mmr_recs[col][0], xquad_recs[col][0], rxquad_recs[col][0]]
    # if gpt_recs[col][0] > baseline_recs[col][0]:
    #     print(col)
results = pd.DataFrame(data=d)

In [141]:
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["baseline (MF)", "Random", "MMR", "xQuAD", "RxQuAD"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2914,0.316022,0.324541,0.220494,0.775826,0.196826,0.398348,baseline (MF)
1,0.1600,0.141438,0.166716,0.106664,0.807197,0.130478,0.420130,Random
2,0.2774,0.303519,0.316902,0.224497,0.862324,0.201087,0.451130,MMR
3,0.2840,0.302617,0.349881,0.212250,0.783798,0.210826,0.452826,xQuAD
4,0.2820,0.297959,0.337899,0.210053,0.772049,0.205522,0.425174,RxQuAD


In [142]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
chat_p1 = pd.read_csv(f"{results_path}/chatgpt0613-div-p1-pzt-fold_0-@10.csv")
chat_p2 = pd.read_csv(f"{results_path}/chatgpt0613-div-p2-pzt-fold_0-@10.csv")
chat_p3 = pd.read_csv(f"{results_path}/chatgpt0613-div-p3-pzt-fold_0-@10.csv")
chat_p4 = pd.read_csv(f"{results_path}/chatgpt0613-div-p4-pzt-fold_0-@10.csv")
chat_p5 = pd.read_csv(f"{results_path}/chatgpt0613-div-p5-pzt-fold_0-@10.csv")
chat_p6 = pd.read_csv(f"{results_path}/chatgpt0613-div-p6-pzt-fold_0-@10.csv")

d = {}
for col in chat_p1.columns:
    d[col] = [chat_p1[col][0], 
              chat_p2[col][0], 
              chat_p3[col][0], 
              chat_p4[col][0],
              chat_p5[col][0],
              chat_p6[col][0],
              ]
results = pd.DataFrame(data=d)
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["chat-gpt-p1", "chat-gpt-p2", "chat-gpt-p3", "chat-gpt-p4", 
                       "chat-gpt-p5", "chat-gpt-p6"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2452,0.270062,0.286624,0.185838,0.812312,0.21485,0.50335,chat-gpt-p1
1,0.2710,0.295994,0.312172,0.211770,0.812039,0.23160,0.50230,chat-gpt-p2
2,0.2540,0.276861,0.299468,0.195332,0.815794,0.22390,0.51065,chat-gpt-p3
3,0.2302,0.251550,0.275403,0.173522,0.817197,0.20865,0.51405,chat-gpt-p4
4,0.2540,0.275740,0.296433,0.193743,0.813080,0.22125,0.50740,chat-gpt-p5
5,0.2506,0.272173,0.296205,0.193616,0.817579,0.22310,0.51355,chat-gpt-p6


In [143]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
instruct_p1 = pd.read_csv(f"{results_path}/instructgpt-div-p1-pzt-fold_0-@10.csv")
instruct_p2 = pd.read_csv(f"{results_path}/instructgpt-div-p2-pzt-fold_0-@10.csv")
instruct_p3 = pd.read_csv(f"{results_path}/instructgpt-div-p3-pzt-fold_0-@10.csv")
instruct_p4 = pd.read_csv(f"{results_path}/instructgpt-div-p4-pzt-fold_0-@10.csv")
instruct_p5 = pd.read_csv(f"{results_path}/instructgpt-div-p5-pzt-fold_0-@10.csv")
instruct_p6 = pd.read_csv(f"{results_path}/instructgpt-div-p6-pzt-fold_0-@10.csv")

d = {}
for col in chat_p1.columns:
    d[col] = [instruct_p1[col][0], 
              instruct_p2[col][0], 
              instruct_p3[col][0], 
              instruct_p4[col][0],
              instruct_p5[col][0],
              instruct_p6[col][0],
              ]
results = pd.DataFrame(data=d)
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["instruct-gpt-p1", "instruct-gpt-p2", "instruct-gpt-p3", "instruct-gpt-p4", 
                       "instruct-gpt-p5", "instruct-gpt-p6"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2430,0.262564,0.286618,0.185676,0.812275,0.21915,0.51010,instruct-gpt-p1
1,0.2454,0.272404,0.291284,0.186974,0.813067,0.21710,0.50565,instruct-gpt-p2
2,0.2372,0.260093,0.281940,0.181968,0.818466,0.21340,0.51370,instruct-gpt-p3
3,0.2394,0.257365,0.283150,0.181650,0.813911,0.21585,0.51060,instruct-gpt-p4
4,0.2466,0.270360,0.292655,0.188584,0.815421,0.21930,0.50480,instruct-gpt-p5
5,0.2462,0.274018,0.294508,0.188309,0.813147,0.21825,0.50360,instruct-gpt-p6


In [144]:
results_path = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval"
llama_p7 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p7-pzt-fold_0-@10.csv")
llama_p8 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p8-pzt-fold_0-@10.csv")
llama_p9 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p9-pzt-fold_0-@10.csv")
llama_p10 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p10-pzt-fold_0-@10.csv")
llama_p11 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p11-pzt-fold_0-@10.csv")
llama_p12 = pd.read_csv(f"{results_path}/Llama-2-7b-chat-hf-div-p12-pzt-fold_0-@10.csv")

d = {}
for col in chat_p1.columns:
    d[col] = [llama_p7[col][0], 
              llama_p8[col][0], 
              llama_p9[col][0], 
              llama_p10[col][0],
              llama_p11[col][0],
              llama_p12[col][0],
              ]
results = pd.DataFrame(data=d)
results = results[["prec", "ndcg", "a_ndcg", "eild_rel", "eild_norel", "s_recall_rel", "s_recall_norel"]]
results["reranker"] = ["llama-p7", "llama-p8", "llama-p9", "llama-p10", 
                       "llama-p11", "llama-p12"]
results

,prec,ndcg,a_ndcg,eild_rel,eild_norel,s_recall_rel,s_recall_norel,reranker
0,0.2296,0.250901,0.277415,0.169752,0.812883,0.21020,0.51205,llama-p7
1,0.2106,0.233646,0.255733,0.156441,0.818012,0.19515,0.50245,llama-p8
2,0.2126,0.234732,0.259793,0.155878,0.818021,0.19620,0.51005,llama-p9
3,0.2318,0.253362,0.281120,0.173538,0.810288,0.21395,0.51190,llama-p10
4,0.2210,0.235086,0.269004,0.164583,0.806263,0.20665,0.51185,llama-p11
5,0.2232,0.241760,0.271037,0.166399,0.805794,0.20610,0.51135,llama-p12


<h2>Is list A more diverse than list B?<h2>

In [7]:
# import df_items
df_anime = pd.read_csv("/home/diego/chat-reranking/experiments/anime/df_items.csv")
df_anime["genres"] = df_anime.genres.apply(literal_eval)
df_anime["n_genres"] = df_anime.apply(lambda row: len(row["genres"]), axis=1)
df_anime.head(1)

,item_id,item_name,pop_score,prem_year,genres,n_genres
0,1535,Death Note,1,2006,"[Mystery, Police, Psychological, Supernatural,...",6


In [14]:
df_few_genres = df_anime[df_anime["n_genres"] == 1]
df_few_genres["only_genre"] = df_few_genres.apply(lambda row: row["genres"][0], axis=1)
print(len(df_few_genres))

128


/tmp/ipykernel_189011/905591025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_few_genres["only_genre"] = df_few_genres.apply(lambda row: row["genres"][0], axis=1)


In [20]:
df_few_genres = df_few_genres[df_few_genres["only_genre"] == "Comedy"]
for i, row in df_few_genres.iterrows():
    print(row["item_name"])

TO BE HERO
Onara Goro
JK-MESHI!
Sparrow's Hotel
Nyanbo!
Scorched Bread
Osomatsu-kun (1988)
Usavich III
.Koni-chan
Usavich IV
100% Teacher Pascal
Princess Beware
How to Breed Kappas
Calimero (1992)
Usavich V
Bonobono
Mr. Stain on Junk Alley
Let's Dance with Papa
King of Classroom, Yamazaki
Poo-Neko
The Adventures of the Little Koala
Oscar's Oasis
The Affectuous Family
Little Wansa
Anyway, It Doesn't Matter
Hippo and Thomas
Weekly Shimako
Table Cat
At that time Mr. Shimakoh moved!
The Adventure of Oden-kun
Tumulus Gal Coffy Campus Life
Yoshimoto Insect Stories


In [21]:
df_few_genres["only_genre"].value_counts()

only_genre
Comedy    32
Name: count, dtype: int64

In [23]:
list_b = ["Hundred",
"The Millionaire Detective – Balance:UNLIMITED",
"Freaky Highschool",
"Arte",
"Infinite Ryvius",
"Is this A Zombie? of the Dead",
"A Christmas Adventure",
"Junjo Romantica",
"Rokka:Braves of the Six Flowers",
"Wooser's Hand-to-Mouth Life:Awakening Arc",
"Jormungand Season 2:Perfect Order"]

for name in list_b:
    print(df_anime[df_anime["item_name"] == name]["genres"].values)

[list(['Action', 'Sci-Fi', 'Harem', 'Romance', 'Ecchi', 'Mecha', 'School'])]
[list(['Mystery', 'Comedy', 'Police'])]
[list(['Horror', 'School'])]
[list(['Drama', 'Historical', 'Romance', 'Seinen', 'Slice of Life'])]
[list(['Drama', 'Mecha', 'Military', 'Psychological', 'Sci-Fi', 'Space'])]
[list(['Action', 'Harem', 'Comedy', 'Supernatural', 'Magic', 'Ecchi'])]
[list(['Fantasy', 'Kids'])]
[list(['Comedy', 'Drama', 'Romance', 'Shounen Ai'])]
[list(['Action', 'Adventure', 'Fantasy', 'Magic', 'Mystery'])]
[list(['Comedy', 'Fantasy', 'Slice of Life'])]
[list(['Action', 'Adventure', 'Seinen'])]


In [24]:
list_a = ["Hundred",
"The Millionaire Detective – Balance:UNLIMITED",
"Freaky Highschool",
"Arte",
"Infinite Ryvius",
"Is this A Zombie? of the Dead",
"A Christmas Adventure",
"Junjo Romantica",
"Rokka:Braves of the Six Flowers",
"Wooser's Hand-to-Mouth Life:Awakening Arc"]

list_b = ["Jormungand Season 2:Perfect Order",
"Genshiken",
"Shirobako",
"Scum's Wish",
"Inuyashiki:Last Hero",
"My Name Is Teppei",
"Magical Project S",
"Hand Shakers",
"Tekkaman Blade",
"Pretty Cure"]

In [33]:
genres_a = set()
df_list_a = df_anime[df_anime["item_name"].isin(list_a)]
name_to_genres_a = {}
for i, row in df_list_a.iterrows():
    genres_a = genres_a.union(row["genres"])
    name_to_genres_a[row["item_name"]] = row["genres"]
print(len(genres_a))

24


In [63]:
print(len(df_list_a))
genres_a

10


{'Action',
 'Adventure',
 'Comedy',
 'Drama',
 'Ecchi',
 'Fantasy',
 'Harem',
 'Historical',
 'Horror',
 'Kids',
 'Magic',
 'Mecha',
 'Military',
 'Mystery',
 'Police',
 'Psychological',
 'Romance',
 'School',
 'Sci-Fi',
 'Seinen',
 'Shounen Ai',
 'Slice of Life',
 'Space',
 'Supernatural'}

In [34]:
genres_b = set()
df_list_b = df_anime[df_anime["item_name"].isin(list_b)]
name_to_genres_b = {}
for i, row in df_list_b.iterrows():
    genres_b = genres_b.union(row["genres"])
    name_to_genres_b[row["item_name"]] = row["genres"]
print(len(genres_b))

19


In [62]:
genres_b

{'Action',
 'Adventure',
 'Comedy',
 'Drama',
 'Fantasy',
 'Magic',
 'Mecha',
 'Military',
 'Parody',
 'Psychological',
 'Romance',
 'School',
 'Sci-Fi',
 'Seinen',
 'Shoujo',
 'Shounen',
 'Slice of Life',
 'Space',
 'Sports'}

In [39]:
def jaccard_dist(ii:list, jj:list) -> float:
    return len(set(ii).intersection(jj))/len(set(ii).union(jj))

In [64]:
combinations_a = itertools.combinations(list_a, 2)
combinations_b = itertools.combinations(list_b, 2)

In [65]:
div_list_a = 0
n_combinations = 0
for (item_1, item_2) in itertools.combinations(list_a, 2):
    jaccard_score = jaccard_dist(name_to_genres_a[item_1], name_to_genres_a[item_2])
    div_list_a += 1 - jaccard_score
    n_combinations += 1
print(div_list_a/n_combinations)

0.925614077280744


In [66]:
div_list_b = 0
n_combinations = 0
for (item_1, item_2) in itertools.combinations(list_b, 2):
    jaccard_score = jaccard_dist(name_to_genres_b[item_1], name_to_genres_b[item_2])
    div_list_b += 1 - jaccard_score
    n_combinations += 1
print(div_list_b/n_combinations)

0.867369482369482


In [ ]:
div_list_a = None
div_list_b = None

print(div_list_a)
print(div_list_b)

<h3>How deep goes re-ranking?<h3>

In [28]:
recs_folder = "/home/diego/chat-reranking/experiments/goodreads/recs/reranked_final"

rec_files = ["chatgpt0613-div-p1-pzt-fold_0",
             "chatgpt0613-div-p2-pzt-fold_0",
             "chatgpt0613-div-p3-pzt-fold_0",
             "chatgpt0613-div-p4-pzt-fold_0",
             "chatgpt0613-div-p5-pzt-fold_0",
             "chatgpt0613-div-p6-pzt-fold_0"
             ]

# rec_files = ["instructgpt-div-p1-pzt-fold_0",
#              "instructgpt-div-p2-pzt-fold_0",
#              "instructgpt-div-p3-pzt-fold_0",
#              "instructgpt-div-p4-pzt-fold_0",
#              "instructgpt-div-p5-pzt-fold_0",
#              "instructgpt-div-p6-pzt-fold_0"
#              ]

# rec_files = ["Llama-2-7b-chat-hf-div-p7-pzt-fold_0",
#              "Llama-2-7b-chat-hf-div-p8-pzt-fold_0",
#              "Llama-2-7b-chat-hf-div-p9-pzt-fold_0",
#              "Llama-2-7b-chat-hf-div-p10-pzt-fold_0",
#              "Llama-2-7b-chat-hf-div-p11-pzt-fold_0",
#              "Llama-2-7b-chat-hf-div-p12-pzt-fold_0",
#              ]

top_n = 10
mean_max_pos = []

for recs_name in rec_files:
    
    # read JSON file
    f = open(f"{recs_folder}/{recs_name}.json")
    data = json.load(f)
    f.close()
    
    max_pos = []
    for entry in data:
        base = entry['recs']
        re_ranked = entry['reranked_recs']
        if len(re_ranked) == 0:
            continue
    
        def return_pos(j):
            if j in base:
                return base.index(j)
            else:
                return 0
    
        pos = [return_pos(r) for r in re_ranked]
        max_pos.append(max(pos))
    
    mean_max_pos.append(np.mean(max_pos))   
    print(f"Done: {recs_name}")

Done: chatgpt0613-div-p1-pzt-fold_0
Done: chatgpt0613-div-p2-pzt-fold_0
Done: chatgpt0613-div-p3-pzt-fold_0
Done: chatgpt0613-div-p4-pzt-fold_0
Done: chatgpt0613-div-p5-pzt-fold_0
Done: chatgpt0613-div-p6-pzt-fold_0


In [29]:
mean_max_pos

[34.768, 25.354, 32.906, 40.27, 26.126, 32.112]

In [30]:
np.mean(mean_max_pos)

31.922666666666668

In [34]:
rerankers = ["MMR-pzt-fold_0", "RxQuAD-pzt-fold_0", "xQuAD-pzt-fold_0"]

print(f"Tune the length of the candidate set: baseline reranker {rerankers[0]}")
base_recs = pd.read_csv(f"/home/diego/chat-reranking/experiments/goodreads/recs/baselines/pzt-fold_0", 
                            names=["userid", "itemid", "rating"], sep="\t")
re_ranked = pd.read_csv(f"/home/diego/chat-reranking/experiments/goodreads/recs/reranked_final/{rerankers[0]}", 
                            names=["userid", "itemid", "rating"], sep="\t")

max_pos = []
for userid in re_ranked["userid"].unique():
    base = list(base_recs[base_recs["userid"] == userid]["itemid"].values.tolist())
    recs = re_ranked[re_ranked["userid"] == userid]["itemid"].values.tolist()
        
    pos = [base.index(r) for r in recs]
    max_pos.append(max(pos))
print(f"Average max reranking pos: {np.mean(max_pos)}")

Tune the length of the candidate set: baseline reranker MMR-pzt-fold_0
Average max reranking pos: 22.578618581099345


In [ ]:
rerankers = ["MMR-pzt-fold_0", "RxQuAD-pzt-fold_0", "xQuAD-pzt-fold_0"]

print(f"Tune the length of the candidate set: baseline reranker {rerankers[1]}")
base_recs = pd.read_csv(f"/home/diego/chat-reranking/experiments/goodreads/recs/baselines/pzt-fold_0", 
                            names=["userid", "itemid", "rating"], sep="\t")
re_ranked = pd.read_csv(f"/home/diego/chat-reranking/experiments/goodreads/recs/reranked_final/{rerankers[1]}", 
                            names=["userid", "itemid", "rating"], sep="\t")

max_pos = []
for userid in re_ranked["userid"].unique():
    base = list(base_recs[base_recs["userid"] == userid]["itemid"].values.tolist())
    recs = re_ranked[re_ranked["userid"] == userid]["itemid"].values.tolist()
        
    pos = [base.index(r) for r in recs]
    max_pos.append(max(pos))
print(f"Average max reranking pos: {np.mean(max_pos)}")

Tune the length of the candidate set: baseline reranker RxQuAD-pzt-fold_0


In [ ]:
rerankers = ["MMR-pzt-fold_0", "RxQuAD-pzt-fold_0", "xQuAD-pzt-fold_0"]

print(f"Tune the length of the candidate set: baseline reranker {rerankers[2]}")
base_recs = pd.read_csv(f"/home/diego/chat-reranking/experiments/goodreads/recs/baselines/pzt-fold_0", 
                            names=["userid", "itemid", "rating"], sep="\t")
re_ranked = pd.read_csv(f"/home/diego/chat-reranking/experiments/goodreads/recs/reranked_final/{rerankers[2]}", 
                            names=["userid", "itemid", "rating"], sep="\t")

max_pos = []
for userid in re_ranked["userid"].unique():
    base = list(base_recs[base_recs["userid"] == userid]["itemid"].values.tolist())
    recs = re_ranked[re_ranked["userid"] == userid]["itemid"].values.tolist()
        
    pos = [base.index(r) for r in recs]
    max_pos.append(max(pos))
print(f"Average max reranking pos: {np.mean(max_pos)}")

<h4>Additional scripts for plots<h4>

In [10]:
results_dir = "/home/diego/chat-reranking/experiments/anime/results/ranksys_eval/"
rankers = ["Llama-2-7b-chat-hf-div-p{}-pzt-fold_0-@10"]
prompts = ["7", "8", "9", "10", "11", "12"]
metrics = ["ndcg","a_ndcg","eild_rel","eild_norel","s_recall_rel","s_recall_norel"]
for ranker in rankers:
    for prompt_id in prompts:
        name = ranker.format(prompt_id)
        # read results file
        df = pd.read_csv(f"{results_dir}/{name}.csv", sep=",")
        df = df[metrics]
        # print(prompt_id)
        print(df.iloc[2].values)
        # print("######")

[0.01731786 0.01780237 0.0138863  0.00422926 0.01207938 0.00528913]
[0.01636567 0.01664738 0.01325456 0.00407414 0.01161214 0.00508864]
[0.01697631 0.01744979 0.01370783 0.00416456 0.01198907 0.00535532]
[0.01710491 0.01773179 0.01362405 0.00437727 0.01207323 0.00536295]
[0.01689876 0.01781902 0.01383339 0.00452346 0.01235096 0.00534317]
[0.01723047 0.0178735  0.01385849 0.00458196 0.01242485 0.00554367]
